### Packages

In [1]:
cd '/h/ama/workspace/ama-at-vector/best-mix'

/scratch/ssd001/home/ama/workspace/ama-at-vector/best-mix


In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import math
import pickle

from mixup import mixup_graph
import time
from utils_mixup import gradmix_v2, gradmix_v2_improved_v3

os.environ['KMP_DUPLICATE_LIB_OK']='True'
%matplotlib inline

from collections import OrderedDict
args={'dataset': 'cifar100', 'data_dir': '/h/ama/workspace/ama-at-vector/best-mix/data', 'root_dir': '/scratch/hdd001/home/ama/mixup/2022-07-09/345324666', 'labels_per_class': 500, 'valid_labels_per_class': 0, 'arch': 'resnext29_4_24', 'initial_channels': 64, 'epochs': 300, 'method': 'ours', 'train': 'ours', 'in_batch': False, 'mixup_alpha': 1.0, 'dropout': False, 'box': False, 'graph': False, 'neigh_size': 4, 'n_labels': 3, 'beta': 1.2, 'gamma': 0.5, 'eta': 0.2, 'transport': True, 't_eps': 0.8, 't_size': -1, 'adv_eps': 10.0, 'adv_p': 0.0, 'clean_lam': 0.0, 'mp': 8, 'batch_size': 100, 'learning_rate': 0.2, 'momentum': 0.9, 'decay': 0.0001, 'schedule': [100, 200], 'gammas': [0.1, 0.1], 'print_freq': 100, 'resume': '', 'start_epoch': 0, 'evaluate': False, 'ngpu': 1, 'workers': 2, 'seed': 2, 'add_name': '', 'log_off': False, 'job_id': 7898345, 'enable_wandb': True, 'wandb_EOT': True, 'wandb_project': 'ours-with-shift-cifar100-resnext-fixed-sampling-update-ratio', 'wandb_log_freq': 10, 'job_name': '345324666', 'blur_sigma': 100.0, 'kernel_size': 5, 'wandb_mode': 'online', 'grad_normalization': 'L1', 'eval_mode': False, 'new_implementation': True, 'with_shift': True, 'use_yp_argmax': False, 'bce_saliency': True, 'rand_pos': 0.1, 'update_ratio': 1.0, 'prob_mix': 1.0, 'mix_schedule': 'fixed', 'mix_scheduled_epoch': 50, 'upper_lambda': 0.7, 'mixup_alpha2': 0.0, 'use_cuda': True}
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict(args)

In [5]:
# args.eval_mode = True
# args.dataset='cifar10'
# args.update_ratio=0.5
args.num_classes=100
# args.blur_sigma=2
# args.rand_pos=0.1
# args.lr=0.2
log = None

In [6]:
''' Model '''
import models
# import torchvision.models as models
from load_data import load_data_subset
from collections import OrderedDict

labels_per_class = 500

batch_size = 100
workers=2
dataset='cifar100'
data_dir='data'
valid_labels_per_class=0
mixup_alpha=0
train_loader, valid_loader, _, test_loader, num_classes = load_data_subset(batch_size,workers,dataset,data_dir,labels_per_class=labels_per_class,valid_labels_per_class=valid_labels_per_class,mixup_alpha=mixup_alpha)

Files already downloaded and verified
Files already downloaded and verified


In [21]:
from __future__ import division
# Codes are borrowed from https://github.com/vikasverma1077/manifold_mixup/tree/master/supervised

import os, sys, shutil, time, random
from collections import OrderedDict

sys.path.append('..')
if sys.version_info[0] < 3:
    import cPickle as pickle
else:
    import _pickle as pickle
import argparse
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
from torch.autograd import Variable
from load_data import load_data_subset
from logger import plotting, copy_script_to_folder, AverageMeter, RecorderMeter, time_string, convert_secs2time
import models
from multiprocessing import Pool

import ipdb
from utils_log import wandbLogger, saveCheckpoint
import torchvision
import torchvision.transforms as transforms
from utils_wip import gradmix, reweighted_lam, gradmix_v2, gradmix_v2_improved, gradmix_v2_improved_v2
from mixup import to_one_hot, get_lambda


def train(train_loader, model, optimizer, epoch, args, log, mp=None):
    '''train given model and dataloader'''
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    mixing_avg = []
    prob_mix = get_prob_mix(args.mix_schedule, args.prob_mix, epoch, args.mix_scheduled_epoch)

    if args.method == 'ours' and args.blur_sigma != 100:
        blurrer = transforms.GaussianBlur(kernel_size=(args.kernel_size, args.kernel_size),
                                          sigma=(args.blur_sigma, args.blur_sigma))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        data_time.update(time.time() - end)
        optimizer.zero_grad()

        input = input.cuda()
        target = target.long().cuda()

        unary = None
        noise = None
        adv_mask1 = 0
        adv_mask2 = 0

        # train with clean images
        # integrating our method :AVery
        if args.train == 'ours':

            batch_size = input.shape[0]
            mix_size = int(batch_size*prob_mix)

            # if mix_size is 0, we are simply doing standard training with no DA
            if mix_size == 0:
                input_var, target_var = Variable(input), Variable(target)
                output, reweighted_target = model(input_var, target_var)

                loss = bce_loss(softmax(output), reweighted_target)
            else:
                if mix_size == batch_size:
                    # entire batch is DA
                    input_2b_mixed = input
                    target_2b_mixed = target
                    input_std = None
                    target_std = None
                else:
                    # some inputs are augmented, some are not
                    input_std, input_2b_mixed = input[:(batch_size-mix_size)], input[(batch_size-mix_size):]
                    target_std, target_2b_mixed = target[:(batch_size-mix_size)], target[(batch_size-mix_size):]

                input_2b_mixed_var = Variable(input_2b_mixed, requires_grad=True)
                target_2b_mixed_var = Variable(target_2b_mixed)

                # calculate saliency
                if args.eval_mode:
                    model.eval()
                    print('in eval mode')
                else:
                    model.train()
                    print('in train mode')

#                 if args.bce_saliency:
                reweighted_target = to_one_hot(target_2b_mixed_var,args.num_classes)
                output = model(input_2b_mixed_var)
                loss_batch_mean = bce_loss(softmax(output), reweighted_target)

                if args.update_ratio != 1.:
                    print('loss_batch mean adjusted')
                    loss_batch_mean *= (1-args.update_ratio)

#                 else:
#                     output = model(input_2b_mixed_var)
#                     if args.use_yp_argmax:
#                         loss_batch = criterion_batch(output, output.argmax(dim=1))
#                     else:
#                         loss_batch = criterion_batch(output, target_2b_mixed_var)

#                     loss_batch_mean = torch.mean(loss_batch, dim=0)

                loss_batch_mean.backward(retain_graph=True)
#                 ipdb.set_trace()

                model.train()

                g = input_2b_mixed_var.grad.data.abs().mean(dim=1, keepdim=True).detach()

                # apply gaussian bluring to the gradients
                if args.blur_sigma == 100:
                    if args.dataset == 'tiny-imagenet-200':
                        _blur_sigma = np.random.uniform(low=2.0, high=3.0)
                    else:
                        _blur_sigma = np.random.uniform(low=1.0, high=2.0)
                    blurrer = transforms.GaussianBlur(kernel_size=(args.kernel_size, args.kernel_size),
                                                      sigma=(_blur_sigma, _blur_sigma))
                g_tilde = blurrer(g)

                if args.mixup_alpha == 0.:
                    sampled_alpha = 0.5
                else:
                    sampled_alpha = get_lambda(args.mixup_alpha)
                sampled_alpha *= args.upper_lambda

                mixed_x, mixed_y, mixed_lam = gradmix_v2_improved_v3(input_2b_mixed_var,
                                                                     target_2b_mixed_var,
                                                                     g_tilde,
                                                                     alpha=sampled_alpha,
                                                                     normalization=args.grad_normalization,
                                                                     debug=False,
                                                                     rand_pos=args.rand_pos)

                if args.update_ratio == 1.:
                    print('zero grad')
                    optimizer.zero_grad()

                reweighted_target_mix = reweighted_lam(mixed_y, mixed_lam, args.num_classes)
                output_mix = model(mixed_x)
                loss = bce_loss(softmax(output_mix), reweighted_target_mix)
                if args.update_ratio != 1.:
                    print('loss  adjusted')
                    loss *= args.update_ratio
                    
                print(loss_batch_mean.item(), loss.item())

    ########

        # measure accuracy and record loss
        if args.train in ['mixup', 'mixup_hidden']:
            if mix_size == 0:
                prec1, prec5 = accuracy(output, target, topk=(1, 5))
            elif mix_size == batch_size:
                prec1, prec5 = accuracy(output_mix, target_2b_mixed, topk=(1, 5))
            else:
                prec1_mix, prec5_mix = accuracy(output_mix, target_2b_mixed, topk=(1, 5))
                prec1_std, prec5_std = accuracy(output_std, target_std, topk=(1, 5))
                prec1 = prec1_mix + prec1_std
                prec5 = prec5_mix + prec5_std
        elif args.train == 'ours':
            if mix_size == 0:
                prec1, prec5 = accuracy(output, target, topk=(1, 5))
            elif mix_size == batch_size:
                prec1, prec5 = accuracy(output_mix, target_2b_mixed, topk=(1, 5))
            else:
                if args.new_implementation:
                    prec1, prec5 = accuracy(output_mix, target_concat, topk=(1, 5))
                else:
                    prec1_mix, prec5_mix = accuracy(output_mix, target_2b_mixed, topk=(1, 5))
                    prec1_std, prec5_std = accuracy(output_std, target_std, topk=(1, 5))
                    prec1 = prec1_mix + prec1_std
                    prec5 = prec5_mix + prec5_std
        elif args.train == 'vanilla':
            prec1, prec5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))

        # compute gradient and do SGD step
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

    if (epoch == 0 or (epoch+1)%args.wandb_log_freq == 0 or (epoch+1) == args.epochs) or args.wandb_EOT:
        print_log(
            '  **Train** Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f} Error@1 {error1:.3f}'.format(
                top1=top1, top5=top5, error1=100 - top1.avg), log)
    return top1.avg, top5.avg, losses.avg

def get_prob_mix(mix_schedule, max_prob, epoch, scheduled_epoch):
    """
    with scheldued mix,
    if epoch < scheduled_epoch, we linearly increase the mix prob from 0 to max_prob
    if epoch > scheduled_epoch, we fix the prob at max_prob
    """
    if mix_schedule == 'fixed':
        prob_mix = max_prob
    elif mix_schedule == 'scheduled':
        if epoch+1>=scheduled_epoch:
            prob_mix = max_prob
        else:
            prob_mix = (epoch+1)/scheduled_epoch*max_prob
    elif mix_schedule == 'delayed':
        if epoch>=scheduled_epoch:
            prob_mix = max_prob
        else:
            prob_mix = 0
    return prob_mix

bce_loss = nn.BCELoss().cuda()
bce_loss_sum = nn.BCELoss(reduction='sum').cuda()
softmax = nn.Softmax(dim=1).cuda()
criterion = nn.CrossEntropyLoss().cuda()
criterion_batch = nn.CrossEntropyLoss(reduction='none').cuda()

def accuracy(output, target, topk=(1, )):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def print_log(print_string, log, end='\n'):
    '''print log'''
    print("{}".format(print_string), end=end)
    if log is not None:
        if end == '\n':
            log.write('{}\n'.format(print_string))
        else:
            log.write('{} '.format(print_string))
        log.flush()

In [26]:
args.arch = 'preactresnet18'
resnet = models.__dict__[args.arch](100, False, 1).cuda()

# checkpoint = torch.load('checkpoint/cifar10_preact_ckpt_vanilla.pth.tar')

# od = OrderedDict()
# for key in checkpoint['state_dict'].keys():
#     od[key[7:]] = checkpoint['state_dict'][key]
# resnet.load_state_dict(od)

# resnet = models.resnet18(pretrained=True)

optimizer = torch.optim.SGD(resnet.parameters(),
                                0.2,
                                momentum=0.9,
                                weight_decay=1e-3,
                                nesterov=True)


In [27]:
args.eval_mode = 1
args.update_ratio = 0.5
args.method = 'ours'
args.train = 'ours'

total_time_list = []
for i in range(1):
    tic = time.perf_counter()
#     try:
    train(train_loader, resnet, optimizer, 0, args, log, mp=None)
#     except:
#         ipdb.set_trace()
    toc = time.perf_counter()
    total_time_list.append(toc-tic)

total_time_list= np.array(total_time_list)
print(f"Average total time: {total_time_list.mean():0.4f} seconds (var: {total_time_list.var():0.4f})")
print(f"Max/Min time: {total_time_list.max():0.4f}/{total_time_list.min():0.4f} seconds")


in eval mode
loss_batch mean adjusted
loss  adjusted
0.027957884594798088 0.028081579133868217
in eval mode
loss_batch mean adjusted
loss  adjusted
0.028034869581460953 0.028320247307419777
in eval mode
loss_batch mean adjusted
loss  adjusted
0.028011035174131393 0.028156865388154984
in eval mode
loss_batch mean adjusted
loss  adjusted
0.027983684092760086 0.028139855712652206
in eval mode
loss_batch mean adjusted
loss  adjusted
0.02799263782799244 0.028008535504341125
in eval mode
loss_batch mean adjusted
loss  adjusted
0.02797263115644455 0.027946028858423233
in eval mode
loss_batch mean adjusted
loss  adjusted
0.028051042929291725 0.028378862887620926
in eval mode
loss_batch mean adjusted
loss  adjusted
0.028050456196069717 0.028182636946439743
in eval mode
loss_batch mean adjusted
loss  adjusted
0.028000609949231148 0.027864113450050354
in eval mode
loss_batch mean adjusted
loss  adjusted
0.027964921668171883 0.028019560500979424
in eval mode
loss_batch mean adjusted


KeyboardInterrupt: 

In [32]:
for i in range(0):
    print(i)